In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

### Extract the crowdfunding.xlsx Data

In [ ]:
# Read the data into a Pandas DataFrame
crowdfunding_info_df = pd.read_excel("../Resources/crowdfunding.xlsx", engine='openpyxl')
crowdfunding_info_df.head()

In [ ]:
# Get a brief summary of the crowdfunding_info DataFrame.
crowdfunding_info_df.info()

### Create the Category and Subcategory DataFrames
---
**Create a Category DataFrame that has the following columns:**
- A "category_id" column that is numbered sequential form 1 to the length of the number of unique categories.
- A "category" column that has only the categories.

Export the DataFrame as a `category.csv` CSV file.

**Create a SubCategory DataFrame that has the following columns:**
- A "subcategory_id" column that is numbered sequential form 1 to the length of the number of unique subcategories.
- A "subcategory" column that has only the subcategories. 

Export the DataFrame as a `subcategory.csv` CSV file.

In [ ]:
# Get the crowdfunding_info_df columns.
columns = crowdfunding_info_df.columns.tolist()
columns

In [ ]:
# Assign the category and subcategory values to category and subcategory columns.
crowdfunding_info_df[['category', 'subcategory']] = crowdfunding_info_df["category & sub-category"].apply(lambda x: pd.Series(str(x).split("/")))
crowdfunding_info_df.head()

In [ ]:
# Get the unique categories and subcategories in separate lists.
categories = list(crowdfunding_info_df['category'].unique())
subcategories = list(crowdfunding_info_df['subcategory'].unique())

print(categories)
print(subcategories)

In [ ]:
# Get the number of distinct values in the categories and subcategories lists.
print(len(categories))
print(len(subcategories))

In [ ]:
# Create numpy arrays from 1-9 for the categories and 1-24 for the subcategories.
category_ids = np.array([1,2,3,4,5,6,7,8,9])
subcategory_ids = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])

print(category_ids)
print(subcategory_ids)

In [ ]:
# Use a list comprehension to add "cat" to each category_id. 

cat_ids = ['cat'+ str(c) for c in category_ids]

# Use a list comprehension to add "subcat" to each subcategory_id.

scat_ids = ['scat'+ str(s) for s in subcategory_ids]
    
print(cat_ids)
print(scat_ids)

In [ ]:
# Create a category DataFrame with the category_id array as the category_id and categories list as the category name.
category_df = pd.DataFrame()
category_df['category_id'] = cat_ids
category_df['category'] = categories
# Create a category DataFrame with the subcategory_id array as the subcategory_id and subcategories list as the subcategory name. 
subcategory_df = pd.DataFrame()
subcategory_df ['subcategory_id'] = scat_ids 
subcategory_df ['subcategory'] = subcategories

In [ ]:
category_df

In [ ]:
subcategory_df

In [ ]:
# Export categories_df and subcategories_df as CSV files.
category_df.to_csv("../Resources/category.csv", index=False)

subcategory_df.to_csv("../Resources/subcategory.csv", index=False)


### Campaign DataFrame
----
**Create a Campaign DataFrame that has the following columns:**
- The "cf_id" column.
- The "contact_id" column.
- The “company_name” column.
- The "blurb" column is renamed as "description."
- The "goal" column.
- The "goal" column is converted to a `float` datatype.
- The "pledged" column is converted to a `float` datatype. 
- The "backers_count" column. 
- The "country" column.
- The "currency" column.
- The "launched_at" column is renamed as "launch_date" and converted to a datetime format. 
- The "deadline" column is renamed as "end_date" and converted to a datetime format.
- The "category_id" with the unique number matching the “category_id” from the category DataFrame. 
- The "subcategory_id" with the unique number matching the “subcategory_id” from the subcategory DataFrame.
- And, create a column that contains the unique four-digit contact ID number from the `contact.xlsx` file.
 

Then export the DataFrame as a `campaign.csv` CSV file.


In [ ]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df. 
campaign_df = crowdfunding_info_df.copy()
campaign_df.head()

In [ ]:
# Rename the blurb, launched_at, and deadline columns.
campaign_df = campaign_df.rename(columns={'blurb': 'description', 'launched_at': 'launched_date', 'deadline': 'end_date'})
campaign_df

In [ ]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df["goal"] = campaign_df["goal"].astype("float")
campaign_df["pledged"] = campaign_df["pledged"].astype("float")

campaign_df 

In [ ]:
# Check the datatypes
campaign_df.dtypes

In [ ]:
# Format the launched_date and end_date columns to datetime format
from datetime import datetime as dt

campaign_df['end_date'] = pd.to_datetime(campaign_df['end_date'])
campaign_df['launched_date'] = pd.to_datetime(campaign_df['launched_date'])


campaign_df


In [ ]:
# Merge the campaign_df with the category_df on the "category" column and 
# the subcategory_df on the "subcategory" column.
campaign_merged_df = pd.DataFrame()
campaign_merged_df = pd.merge(campaign_df,category_df, how='outer', on='category')
campaign_merged_df = pd.merge(campaign_merged_df,subcategory_df, how='outer', on='subcategory')

campaign_merged_df.tail(10)

In [ ]:
# Drop unwanted columns
campaign_cleaned = campaign_merged_df.drop(['staff_pick','spotlight','category & sub-category','category','subcategory'],axis=1)
campaign_merged_df.tail(10)

In [ ]:
# Export the DataFrame as a CSV file. 
campaign_cleaned.to_csv("../Resources/campaign.csv", index=False)

### Extract the contacts.xlsx Data.

In [ ]:
# Read the data into a Pandas DataFrame. Use the `header=2` parameter when reading in the data.
contact_info_df = pd.read_excel('../Resources/contacts.xlsx', engine='openpyxl', header=2)
contact_info_df.head()

### Create the Contacts DataFrame 
---
**Create a Contacts DataFrame that has the following columns:**
- A column named "contact_id"  that contains the unique number of the contact person.
- A column named "first_name" that contains the first name of the contact person.
- A column named "last_name" that contains the first name of the contact person.
- A column named "email" that contains the email address of the contact person

Then export the DataFrame as a `contacts.csv` CSV file.

### Option 1: Use Pandas to create the contacts DataFrame.

In [ ]:
# Iterate through the contact_info_df and convert each row to a dictionary.
import json
dict_values = []

for i, row in contact_info_df.iterrows():
    data=row['contact_info']
    rows=json.loads(data)
    values=[]
    values.append(rows['contact_id'])
    values.append(rows['name'])
    values.append(rows['email'])
    dict_values.append(values)

# Print out the list of values for each row.
print(dict_values)

In [ ]:
# Create a contact_info DataFrame and add each list of values, i.e., each row 
# to the 'contact_id', 'name', 'email' columns.
contact_info = pd.DataFrame(dict_values, columns = ['contact_id','name','email'])
contact_info.head()

In [ ]:
# Check the datatypes.
contact_info.info()

In [ ]:
# Create a "first"name" and "last_name" column with the first and last names from the "name" column. 
contact_info[["first_name","last_name"]] = contact_info["name"].str.split(' ', n=1, expand=True)
contact_info.head(10)

# Drop the contact_name column
contact_info_dropped = contact_info.drop(columns=['name'])
contact_info_dropped.head(10)


In [ ]:
# Reorder the columns
contacts_df_clean = contact_info_dropped[['contact_id','first_name','last_name','email']]
contacts_df_clean.head(10)

In [ ]:
# Check the datatypes one more time before exporting as CSV file.
contacts_df_clean.info()

In [ ]:
# Export the DataFrame as a CSV file. 
contacts_df_clean.to_csv("../Resources/contacts.csv", encoding='utf8', index=False)

### Option 2: Use regex to create the contacts DataFrame.

In [ ]:
contact_info_df_copy = contact_info_df.copy()
contact_info_df_copy.head()

In [ ]:
# Extract the four-digit contact ID number.
contact_info_df_copy['contact_id'] = contact_info_df_copy['contact_info'].str.extract(r'contact_id": (\d{4})')
contact_info_df_copy

In [ ]:
# Check the datatypes.
contact_info_df_copy.info()

In [ ]:
# Convert the "contact_id" column to an int64 data type.
contact_info_df_copy['contact_id']=contact_info_df_copy['contact_id'].astype(int)
contact_info_df_copy.info()

In [ ]:
# Extract the name of the contact and add it to a new column.
contact_info_df_copy['name'] = contact_info_df_copy['contact_info'].str.extract(r'"name": "([^"]*)"')
contact_info_df_copy

In [ ]:
# Extract the email from the contacts and add the values to a new column.
contact_info_df_copy['email'] = contact_info_df_copy['contact_info'].str.extract(r'"email": "([^"]*)"')
contact_info_df_copy

In [ ]:
# Create a copy of the contact_info_df with the 'contact_id', 'name', 'email' columns.
contact_info_df_copy=contact_info_df_copy[['contact_id','name', 'email']]
contact_info_df_copy

In [ ]:
# Create a "first"name" and "last_name" column with the first and last names from the "name" column. 
contact_info_df_copy['first_name']=contact_info_df_copy['name'].str.extract(r'(\w+)')
contact_info_df_copy['last_name']=contact_info_df_copy['name'].str.extract(r'(?:\s)(\w+)')

# Drop the contact_name column
del contact_info_df_copy['name']

In [ ]:
# Reorder the columns
contact_info_df_copy=contact_info_df_copy[['contact_id', 'first_name', 'last_name' , 'email']]

In [ ]:
# Check the datatypes one more time before exporting as CSV file.
contact_info_df_copy.info()

In [ ]:
contact_info_df_copy

In [ ]:
# Export the DataFrame as a CSV file. 
contacts_df_clean.to_csv("../Resources/contacts.csv", encoding='utf8', index=False)